In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

In [2]:
def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because th dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset

In [3]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset, verbose=0)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

In [4]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs

# Experiment 1: standard neural network

In [5]:
def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [6]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)

In [7]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

Start training the model...
Model training finished.
Train RMSE: 0.751
Evaluating model performance...
Test RMSE: 0.745


In [8]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

Predicted: 5.3 - Actual: 6.0
Predicted: 6.2 - Actual: 7.0
Predicted: 6.2 - Actual: 7.0
Predicted: 6.4 - Actual: 6.0
Predicted: 6.1 - Actual: 6.0
Predicted: 5.0 - Actual: 5.0
Predicted: 5.6 - Actual: 5.0
Predicted: 6.1 - Actual: 6.0
Predicted: 5.8 - Actual: 5.0
Predicted: 6.0 - Actual: 5.0


# Experiment 2: Bayesian neural network (BNN)

In [9]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

In [11]:
num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)


Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


Start training the model...
Model training finished.
Train RMSE: 0.797
Evaluating model performance...
Test RMSE: 0.798


In [10]:
def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [12]:
def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )


compute_predictions(bnn_model_small)

Predictions mean: 5.33, min: 4.66, max: 6.05, range: 1.4 - Actual: 6.0
Predictions mean: 6.28, min: 5.97, max: 6.56, range: 0.6 - Actual: 7.0
Predictions mean: 6.22, min: 5.83, max: 6.49, range: 0.66 - Actual: 7.0
Predictions mean: 6.36, min: 5.91, max: 6.58, range: 0.67 - Actual: 6.0
Predictions mean: 6.15, min: 5.84, max: 6.43, range: 0.59 - Actual: 6.0
Predictions mean: 5.1, min: 4.43, max: 6.0, range: 1.58 - Actual: 5.0
Predictions mean: 5.66, min: 5.15, max: 6.26, range: 1.11 - Actual: 5.0
Predictions mean: 6.15, min: 5.84, max: 6.43, range: 0.59 - Actual: 6.0
Predictions mean: 5.81, min: 5.1, max: 6.36, range: 1.27 - Actual: 5.0
Predictions mean: 6.2, min: 5.95, max: 6.45, range: 0.5 - Actual: 5.0


## Train BNN with the whole training set.

In [13]:
num_epochs = 500
bnn_model_full = create_bnn_model(train_size)
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)

compute_predictions(bnn_model_full)

Start training the model...
Model training finished.
Train RMSE: 0.765
Evaluating model performance...
Test RMSE: 0.778
Predictions mean: 5.29, min: 5.0, max: 5.59, range: 0.58 - Actual: 6.0
Predictions mean: 6.35, min: 5.96, max: 6.53, range: 0.57 - Actual: 7.0
Predictions mean: 6.22, min: 5.75, max: 6.43, range: 0.68 - Actual: 7.0
Predictions mean: 6.51, min: 6.01, max: 6.64, range: 0.64 - Actual: 6.0
Predictions mean: 6.1, min: 5.71, max: 6.35, range: 0.63 - Actual: 6.0
Predictions mean: 5.17, min: 4.88, max: 5.42, range: 0.53 - Actual: 5.0
Predictions mean: 5.67, min: 5.3, max: 5.94, range: 0.64 - Actual: 5.0
Predictions mean: 6.12, min: 5.73, max: 6.35, range: 0.62 - Actual: 6.0
Predictions mean: 5.74, min: 5.36, max: 6.01, range: 0.65 - Actual: 5.0
Predictions mean: 6.13, min: 5.71, max: 6.32, range: 0.61 - Actual: 5.0


# Experiment 3: probabilistic Bayesian neural network

In [14]:
def create_probablistic_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # Create a probabilisticå output (Normal distribution), and use the `Dense` layer
    # to produce the parameters of the distribution.
    # We set units=2 to learn both the mean and the variance of the Normal distribution.
    distribution_params = layers.Dense(units=2)(features)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [15]:
def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)


num_epochs = 1000
prob_bnn_model = create_probablistic_bnn_model(train_size)
run_experiment(prob_bnn_model, negative_loglikelihood, train_dataset, test_dataset)

Start training the model...
Model training finished.
Train RMSE: 1.064
Evaluating model performance...
Test RMSE: 1.085


In [16]:
prediction_distribution = prob_bnn_model(examples)
prediction_mean = prediction_distribution.mean().numpy().tolist()
prediction_stdv = prediction_distribution.stddev().numpy()

# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()

for idx in range(sample):
    print(
        f"Prediction mean: {round(prediction_mean[idx][0], 2)}, "
        f"stddev: {round(prediction_stdv[idx][0], 2)}, "
        f"95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}]"
        f" - Actual: {targets[idx]}"
    )

Prediction mean: 5.48, stddev: 0.72, 95% CI: [6.89 - 4.06] - Actual: 6.0
Prediction mean: 6.25, stddev: 0.8, 95% CI: [7.82 - 4.68] - Actual: 7.0
Prediction mean: 6.06, stddev: 0.74, 95% CI: [7.52 - 4.6] - Actual: 7.0
Prediction mean: 6.43, stddev: 0.8, 95% CI: [7.99 - 4.86] - Actual: 6.0
Prediction mean: 5.87, stddev: 0.72, 95% CI: [7.28 - 4.47] - Actual: 6.0
Prediction mean: 5.22, stddev: 0.69, 95% CI: [6.57 - 3.87] - Actual: 5.0
Prediction mean: 5.62, stddev: 0.7, 95% CI: [6.99 - 4.24] - Actual: 5.0
Prediction mean: 5.9, stddev: 0.72, 95% CI: [7.31 - 4.49] - Actual: 6.0
Prediction mean: 5.63, stddev: 0.72, 95% CI: [7.04 - 4.22] - Actual: 5.0
Prediction mean: 5.85, stddev: 0.74, 95% CI: [7.3 - 4.39] - Actual: 5.0
